In [ ]:
!pip install langchain \
              openai==0.28.0 \
              tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from google.colab import userdata
import openai

In [ ]:
import pandas as pd

data = pd.read_csv('knowledgebase1k.csv',encoding='Windows-1252')
data['text'][0]

"User: I'm looking for a hotel called the a and b guest house. Can you help me out? Assistant: Yes, what would you like to know about it? User: I'm hoping to book a room for 8 people on Friday. We will be in town for 4 nights. Assistant: I was able to successfully book that for you. Your reference number is XUSQ4E7T . Is there anything else I can do for you today? User: Yes, I need a train that leaves after 9:30 Assistant: I can help with that. What are your departure and arrival stations please? User: I am departing Cambridge and going to Peterborough, on Tuesday please."

In [ ]:
#
#try this script if above doesn't work
import csv

data_dict = {}
with open('knowledgebase.csv', 'r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        data_dict[row['title']] = row['text']

print(data_dict)

In [ ]:
import chardet
#to get the encoding of the csv file. Check if there's parsing error
with open('knowledgebase.csv', 'rb') as f:
    encoding = chardet.detect(f.read())['encoding']

print(f"Encoding of the CSV file: {encoding}")

Encoding of the CSV file: Windows-1252


In [ ]:
from langchain.chat_models import ChatOpenAI
from google.colab import userdata

chat = ChatOpenAI(
    openai_api_key=userdata.get("OpenAI_API_KEY"),
    model='gpt-3.5-turbo-0125'
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant who will generate meaningfull question-answer pair and will summerize some passages"),
]

In [ ]:
total_messages = messages

In [ ]:
def augment_prompt_for_qa(row: str):
    # get top 3 results from knowledge base
    # get the text from the results
    # feed into an augmented prompt
    augmented_prompt = f""" Generate 1 Question Answer pairs from the following {row}. Answer should be maximun 5 words.
                            The question should be generic. Avoid interrogative answers.
                            \n\n You should create those in the following format \n\n Q: (here should be the question) \n A: (here should be the answer)"""
    return augmented_prompt
def augment_prompt_for_summarize(passage: str):
   augmented_prompt = f""" summarize the {passage} in one sentence."""
   return augmented_prompt

In [ ]:
import csv
with open('QAPairs.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['KB','source (q)', 'target (a)', 'source (reconst)', 'target (reconst)'])

In [ ]:
import re
import csv
from datetime import datetime
pattern = r'Q: (.*?)\nA: (.*)'
print(datetime.now())

with open('QAPairs.csv', 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)

    for i in range(len(data)):
        row = data['text'][i]
        print(i)
        prompt = HumanMessage(content=augment_prompt_for_qa(row))
        # total_messages.append(augment_prompt_for_qa(row))
        res = chat([prompt])
        # total_messages.append(res.content)
        matches = re.findall(pattern, res.content, re.DOTALL)
        source_recon = ""
        summary_res_content = ""
        # print(res.content)
        # print(matches)
        if len(row.split()) > 45:
          summary_prompt = HumanMessage(content=augment_prompt_for_summarize(row))
          summary_res = chat([summary_prompt])
          source_recon = "<p> " + summary_res.content
          summary_res_content = summary_res.content


        for match in matches:
          # print(row, [match[0], match[1]], source_recon, summary_res.content)
          writer.writerow([row, match[0], match[1], source_recon, summary_res_content])


print("Data has been appended to QAPairs.csv")
print(datetime.now())

2024-05-09 03:37:39.768622
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270

In [ ]:
print(total_messages)

[SystemMessage(content='You are a helpful assistant who will generate meaningfull question-answer pair and will summerize some passages'), " Generate 1 Question Answer pairs from the following User: i need a place to dine in the center thats expensive Assistant: I have several options for you; do you prefer African, Asian, or British food? User: Any sort of food would be fine, as long as it is a bit expensive. Could I get the phone number for your recommendation? Assistant: There is an Afrian place named Bedouin in the centre. How does that sound? User: Sounds good, could I get that phone number? Also, could you recommend me an expensive hotel? Assistant: Bedouin's phone is 01223367660. As far as hotels go, I recommend the University Arms Hotel in the center of town.. Answer should be maximun 5 words.\n                            The question should be generic. Avoid interrogative answers.\n                            \n\n You should create those in the following format \n\n Q: (here s

In [ ]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0125"):
  """Returns the number of tokens used by a list of messages."""
  import tiktoken
  try:
      encoding = tiktoken.encoding_for_model(model)
  except KeyError:
      encoding = tiktoken.get_encoding("cl100k_base")
  if model == "gpt-3.5-turbo-0125":  # note: future models may deviate from this
      num_tokens = 0
      for message in messages:
          num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
          for key, value in message.items():
              num_tokens += len(encoding.encode(value))
              if key == "name":  # if there's a name, the role is omitted
                  num_tokens += -1  # role is always required and always 1 token
      num_tokens += 2  # every reply is primed with <im_start>assistant
      return num_tokens
  else:
      raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

In [ ]:
print(num_tokens_from_messages(total_messages,"gpt-3.5-turbo-0125"))

AttributeError: 'SystemMessage' object has no attribute 'items'

In [ ]:
messages=[
    {"role": "system", "content": prompt},
    # {"role": "user", "content": "Who won the world series in 2020?"},
    # {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": query}
  ]